In [1]:
K_q = lambda K_mathcal , q : [idx for idx,K in enumerate(K_mathcal) if q in K]

prob_K_q = lambda K_mathcal, q, prob : sum([prob[idx] for idx in K_q(K_mathcal , q)])

In [2]:
import numpy as np

# Q is the set of items
# K_mathcal is the knowledge network
# K0 is the set of items that the student understands. We use it for simulations but not for real student.
# beta_q is the probability for the student to fail to response item q despite the fact that he manages this item.

class knowledge_learning_space:
    
    def __init__(self,Q,K_mathcal,Xi,beta_q=None,prob_distribution = None,K0=None,student_name = None):

        
        if student_name != None:
            self.student_name = student_name
        
        if (beta_q == None) and (K0!= None):
            self.beta_q = {q : 0.0 for q in K0}
        else:    
            self.beta_q = beta_q
        
        self.K0 = K0
        self.Xi = Xi
        self.Q = Q
        self.K_mathcal = K_mathcal
        n_K = len(K_mathcal)
        
        if prob_distribution == None:
            
            self.prob_distribution =[1/n_K for K in K_mathcal] 
            
        else:
            
            self.prob_distribution = prob_distribution
            
        self.prob_distribution_items = {q : prob_K_q(K_mathcal, q,self.prob_distribution) for q in self.Q}    
        self.min_half = min([np.abs(p - 0.5) for p in self.prob_distribution_items.values()])    
        self.distance_half = [q for q in self.Q if np.abs(self.prob_distribution_items[q]  -0.5 ) \
                              == self.min_half]
        
        self.responded_items = {q : [] for q in Q}
        
        idx_next_item = np.random.randint(0,len(self.distance_half))
        
        self.next_item = self.distance_half[idx_next_item]
        
    def simulation(self):
        
        if self.K0 == None:
            raise NameError('No se puede simular sin conocer K0')
        
        q = self.next_item
        r = int((q in self.K0) and (np.random.rand() <= 1 - self.beta_q[q]))
        self.responded_items[q].append(r)
        
        for idx,K in enumerate(self.K_mathcal):
            
            if int(q in K) == r:
                
                self.prob_distribution[idx] = self.Xi[(q,r)] * self.prob_distribution[idx]
                
        S = np.sum(self.prob_distribution)        
                
        for idx in range(len(self.K_mathcal)):     
                
                self.prob_distribution[idx] = self.prob_distribution[idx] / S
                
        self.prob_distribution_items = {q : prob_K_q(K_mathcal, q,self.prob_distribution) for q in self.Q}    
        
        self.min_half = min([np.abs(p - 0.5) for p in self.prob_distribution_items.values()])    
            
        self.distance_half = [q for q in self.Q if np.abs(self.prob_distribution_items[q]  -0.5 ) \
                              == self.min_half]
        
        idx_next_item = np.random.randint(0,len(self.distance_half))
        
        self.next_item = self.distance_half[idx_next_item]
        
    def response_item(self,r):
        
        q = self.next_item
        self.responded_items[q].append(r)    
        
        for idx,K in enumerate(self.K_mathcal):
            
            if int(q in K) == r:
                
                self.prob_distribution[idx] = self.Xi[(q,r)] * self.prob_distribution[idx]
                
        S = np.sum(self.prob_distribution)        
                
        for idx in range(len(self.K_mathcal)):     
                
                self.prob_distribution[idx] = self.prob_distribution[idx] / S
                
        self.prob_distribution_items = {q : prob_K_q(K_mathcal, q,self.prob_distribution) for q in Q}    
        self.min_half = min([np.abs(p - 0.5) for p in self.prob_distribution_items.values()])    
        self.distance_half = [q for q in self.Q if np.abs(self.prob_distribution_items[q]  -0.5 ) \
                              == self.min_half]
        
        idx_next_item = np.random.randint(0,len(self.distance_half))
        self.next_item = self.distance_half[idx_next_item]
        
    def info_preguntas_respondidas(self):
        
        self.nb_preguntas_respondidas = sum([len(pregunta) for pregunta in self.responded_items.values()])
        self.nb_respuestas_correctas = sum([sum(pregunta) for pregunta in self.responded_items.values()])
        self.nb_respuestas_incorrectas = self.nb_preguntas_respondidas - self.nb_respuestas_correctas

# Example 1: simulación

In [3]:
Q = ['a' , 'b' , 'c' , 'd' , 'e']
K_mathcal = [[],['a'],['b'],['a','b'],['b','d'],['a','b','c'],['a','b','d'],['b','c','d'],Q]
K0 = ['b','d']
beta_q = {'b' : 0.1 ,'d' : 0.1}

QxR = []

for q in Q:
    for r in [0,1]:
        QxR.append((q,r))

Xi = {x : 4.0 for x in QxR}

In [4]:
network_learning = knowledge_learning_space(Q=Q,K_mathcal=K_mathcal,Xi=Xi,beta_q=beta_q,K0=K0)

In [5]:
network_learning.responded_items

{'a': [], 'b': [], 'c': [], 'd': [], 'e': []}

Probabilidad de cada estado de conocimiento:

In [6]:
for knowledge_state, prob in zip(network_learning.K_mathcal,network_learning.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.1111111111111111
Probability(['a'])=0.1111111111111111
Probability(['b'])=0.1111111111111111
Probability(['a', 'b'])=0.1111111111111111
Probability(['b', 'd'])=0.1111111111111111
Probability(['a', 'b', 'c'])=0.1111111111111111
Probability(['a', 'b', 'd'])=0.1111111111111111
Probability(['b', 'c', 'd'])=0.1111111111111111
Probability(['a', 'b', 'c', 'd', 'e'])=0.1111111111111111


Probabilidad que el estudiante maneje cada item:

In [7]:
for item, prob in network_learning.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.5555555555555556
Probability(b)=0.7777777777777779
Probability(c)=0.3333333333333333
Probability(d)=0.4444444444444444
Probability(e)=0.1111111111111111


In [8]:
network_learning.next_item

'a'

In [9]:
network_learning.simulation()

In [10]:
network_learning.responded_items

{'a': [0], 'b': [], 'c': [], 'd': [], 'e': []}

In [11]:
for knowledge_state, prob in zip(network_learning.K_mathcal,network_learning.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.19047619047619047
Probability(['a'])=0.047619047619047616
Probability(['b'])=0.19047619047619047
Probability(['a', 'b'])=0.047619047619047616
Probability(['b', 'd'])=0.19047619047619047
Probability(['a', 'b', 'c'])=0.047619047619047616
Probability(['a', 'b', 'd'])=0.047619047619047616
Probability(['b', 'c', 'd'])=0.19047619047619047
Probability(['a', 'b', 'c', 'd', 'e'])=0.047619047619047616


In [12]:
for item, prob in network_learning.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.23809523809523808
Probability(b)=0.7619047619047619
Probability(c)=0.2857142857142857
Probability(d)=0.47619047619047616
Probability(e)=0.047619047619047616


In [13]:
network_learning.next_item

'd'

In [14]:
network_learning.simulation()

In [15]:
network_learning.responded_items

{'a': [0], 'b': [], 'c': [], 'd': [1], 'e': []}

In [16]:
for knowledge_state, prob in zip(network_learning.K_mathcal,network_learning.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.07843137254901959
Probability(['a'])=0.019607843137254898
Probability(['b'])=0.07843137254901959
Probability(['a', 'b'])=0.019607843137254898
Probability(['b', 'd'])=0.31372549019607837
Probability(['a', 'b', 'c'])=0.019607843137254898
Probability(['a', 'b', 'd'])=0.07843137254901959
Probability(['b', 'c', 'd'])=0.31372549019607837
Probability(['a', 'b', 'c', 'd', 'e'])=0.07843137254901959


In [17]:
for item, prob in network_learning.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.21568627450980388
Probability(b)=0.9019607843137252
Probability(c)=0.41176470588235287
Probability(d)=0.784313725490196
Probability(e)=0.07843137254901959


In [18]:
network_learning.simulation()

In [19]:
network_learning.responded_items

{'a': [0], 'b': [], 'c': [0], 'd': [1], 'e': []}

In [20]:
for knowledge_state, prob in zip(network_learning.K_mathcal,network_learning.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.11347517730496454
Probability(['a'])=0.028368794326241134
Probability(['b'])=0.11347517730496454
Probability(['a', 'b'])=0.028368794326241134
Probability(['b', 'd'])=0.45390070921985815
Probability(['a', 'b', 'c'])=0.0070921985815602835
Probability(['a', 'b', 'd'])=0.11347517730496454
Probability(['b', 'c', 'd'])=0.11347517730496454
Probability(['a', 'b', 'c', 'd', 'e'])=0.028368794326241134


In [21]:
for item, prob in network_learning.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.2056737588652482
Probability(b)=0.8581560283687943
Probability(c)=0.14893617021276595
Probability(d)=0.7092198581560283
Probability(e)=0.028368794326241134


In [22]:
network_learning.simulation()

In [23]:
network_learning.responded_items

{'a': [0], 'b': [], 'c': [0], 'd': [1, 1], 'e': []}

In [24]:
for knowledge_state, prob in zip(network_learning.K_mathcal,network_learning.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.036281179138321996
Probability(['a'])=0.009070294784580499
Probability(['b'])=0.036281179138321996
Probability(['a', 'b'])=0.009070294784580499
Probability(['b', 'd'])=0.5804988662131519
Probability(['a', 'b', 'c'])=0.0022675736961451248
Probability(['a', 'b', 'd'])=0.14512471655328799
Probability(['b', 'c', 'd'])=0.14512471655328799
Probability(['a', 'b', 'c', 'd', 'e'])=0.036281179138321996


In [25]:
for item, prob in network_learning.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.2018140589569161
Probability(b)=0.9546485260770975
Probability(c)=0.1836734693877551
Probability(d)=0.9070294784580498
Probability(e)=0.036281179138321996


In [26]:
network_learning.simulation()

In [27]:
network_learning.responded_items

{'a': [0, 0], 'b': [], 'c': [0], 'd': [1, 1], 'e': []}

In [28]:
for knowledge_state, prob in zip(network_learning.K_mathcal,network_learning.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.04275217100868403
Probability(['a'])=0.002672010688042752
Probability(['b'])=0.04275217100868403
Probability(['a', 'b'])=0.002672010688042752
Probability(['b', 'd'])=0.6840347361389445
Probability(['a', 'b', 'c'])=0.000668002672010688
Probability(['a', 'b', 'd'])=0.04275217100868403
Probability(['b', 'c', 'd'])=0.17100868403473612
Probability(['a', 'b', 'c', 'd', 'e'])=0.010688042752171008


In [29]:
for item, prob in network_learning.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.05945223780895123
Probability(b)=0.9545758183032732
Probability(c)=0.18236472945891782
Probability(d)=0.9084836339345357
Probability(e)=0.010688042752171008


In [30]:
for item, prob in network_learning.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.05945223780895123
Probability(b)=0.9545758183032732
Probability(c)=0.18236472945891782
Probability(d)=0.9084836339345357
Probability(e)=0.010688042752171008


In [31]:
network_learning.info_preguntas_respondidas()

print(f"El estudiante ha respondido a {network_learning.nb_preguntas_respondidas} preguntas, \
tuvo {network_learning.nb_respuestas_correctas} correctas y {network_learning.nb_respuestas_incorrectas}\
 incorrectas.")

El estudiante ha respondido a 5 preguntas, tuvo 2 correctas y 3 incorrectas.


# Example 2: response of student

In [32]:
Q = ['a' , 'b' , 'c' , 'd' , 'e']
K_mathcal = [[],['a'],['b'],['a','b'],['b','d'],['a','b','c'],['a','b','d'],['b','c','d'],Q]
nombre = "Paula"

QxR = []

for q in Q:
    for r in [0,1]:
        QxR.append((q,r))

Xi = {x : 4.0 for x in QxR}

In [33]:
Paula =  knowledge_learning_space(Q,K_mathcal,Xi,student_name = nombre)

In [34]:
Paula.responded_items

{'a': [], 'b': [], 'c': [], 'd': [], 'e': []}

Asumimos que Paula sabe responder correctamente a los items a y b

In [35]:
for knowledge_state, prob in zip(Paula.K_mathcal,Paula.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.1111111111111111
Probability(['a'])=0.1111111111111111
Probability(['b'])=0.1111111111111111
Probability(['a', 'b'])=0.1111111111111111
Probability(['b', 'd'])=0.1111111111111111
Probability(['a', 'b', 'c'])=0.1111111111111111
Probability(['a', 'b', 'd'])=0.1111111111111111
Probability(['b', 'c', 'd'])=0.1111111111111111
Probability(['a', 'b', 'c', 'd', 'e'])=0.1111111111111111


In [36]:
for item, prob in Paula.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.5555555555555556
Probability(b)=0.7777777777777779
Probability(c)=0.3333333333333333
Probability(d)=0.4444444444444444
Probability(e)=0.1111111111111111


In [37]:
Paula.next_item

'a'

In [38]:
Paula.response_item(1)

In [39]:
Paula.responded_items

{'a': [1], 'b': [], 'c': [], 'd': [], 'e': []}

In [40]:
for knowledge_state, prob in zip(Paula.K_mathcal,Paula.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.04166666666666666
Probability(['a'])=0.16666666666666663
Probability(['b'])=0.04166666666666666
Probability(['a', 'b'])=0.16666666666666663
Probability(['b', 'd'])=0.04166666666666666
Probability(['a', 'b', 'c'])=0.16666666666666663
Probability(['a', 'b', 'd'])=0.16666666666666663
Probability(['b', 'c', 'd'])=0.04166666666666666
Probability(['a', 'b', 'c', 'd', 'e'])=0.16666666666666663


In [41]:
for item, prob in Paula.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.8333333333333331
Probability(b)=0.7916666666666665
Probability(c)=0.3749999999999999
Probability(d)=0.4166666666666666
Probability(e)=0.16666666666666663


In [42]:
Paula.next_item

'd'

In [43]:
Paula.response_item(0)

In [44]:
Paula.responded_items

{'a': [1], 'b': [], 'c': [], 'd': [0], 'e': []}

In [45]:
for knowledge_state, prob in zip(Paula.K_mathcal,Paula.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.060606060606060615
Probability(['a'])=0.24242424242424246
Probability(['b'])=0.060606060606060615
Probability(['a', 'b'])=0.24242424242424246
Probability(['b', 'd'])=0.015151515151515154
Probability(['a', 'b', 'c'])=0.24242424242424246
Probability(['a', 'b', 'd'])=0.060606060606060615
Probability(['b', 'c', 'd'])=0.015151515151515154
Probability(['a', 'b', 'c', 'd', 'e'])=0.060606060606060615


In [46]:
for item, prob in Paula.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.8484848484848487
Probability(b)=0.6969696969696971
Probability(c)=0.31818181818181823
Probability(d)=0.15151515151515155
Probability(e)=0.060606060606060615


In [47]:
Paula.next_item

'c'

In [48]:
Paula.response_item(0)

In [49]:
Paula.responded_items

{'a': [1], 'b': [], 'c': [0], 'd': [0], 'e': []}

In [50]:
for knowledge_state, prob in zip(Paula.K_mathcal,Paula.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.07960199004975124
Probability(['a'])=0.31840796019900497
Probability(['b'])=0.07960199004975124
Probability(['a', 'b'])=0.31840796019900497
Probability(['b', 'd'])=0.01990049751243781
Probability(['a', 'b', 'c'])=0.07960199004975124
Probability(['a', 'b', 'd'])=0.07960199004975124
Probability(['b', 'c', 'd'])=0.004975124378109453
Probability(['a', 'b', 'c', 'd', 'e'])=0.01990049751243781


In [51]:
for item, prob in Paula.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.8159203980099502
Probability(b)=0.6019900497512437
Probability(c)=0.1044776119402985
Probability(d)=0.12437810945273631
Probability(e)=0.01990049751243781


In [52]:
Paula.next_item

'b'

In [53]:
Paula.response_item(1)

In [54]:
Paula.responded_items

{'a': [1], 'b': [1], 'c': [0], 'd': [0], 'e': []}

In [55]:
for knowledge_state, prob in zip(Paula.K_mathcal,Paula.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.028368794326241138
Probability(['a'])=0.11347517730496455
Probability(['b'])=0.11347517730496455
Probability(['a', 'b'])=0.4539007092198582
Probability(['b', 'd'])=0.028368794326241138
Probability(['a', 'b', 'c'])=0.11347517730496455
Probability(['a', 'b', 'd'])=0.11347517730496455
Probability(['b', 'c', 'd'])=0.007092198581560284
Probability(['a', 'b', 'c', 'd', 'e'])=0.028368794326241138


In [56]:
for item, prob in Paula.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.8226950354609931
Probability(b)=0.8581560283687946
Probability(c)=0.14893617021276598
Probability(d)=0.17730496453900713
Probability(e)=0.028368794326241138


In [57]:
Paula.next_item

'd'

In [58]:
Paula.response_item(0)

In [59]:
for knowledge_state, prob in zip(Paula.K_mathcal,Paula.prob_distribution):
    print(f'Probability({knowledge_state})={prob}')

Probability([])=0.032719836400818
Probability(['a'])=0.130879345603272
Probability(['b'])=0.130879345603272
Probability(['a', 'b'])=0.523517382413088
Probability(['b', 'd'])=0.0081799591002045
Probability(['a', 'b', 'c'])=0.130879345603272
Probability(['a', 'b', 'd'])=0.032719836400818
Probability(['b', 'c', 'd'])=0.002044989775051125
Probability(['a', 'b', 'c', 'd', 'e'])=0.0081799591002045


In [60]:
for item, prob in Paula.prob_distribution_items.items():
    print(f'Probability({item})={prob}')

Probability(a)=0.8261758691206544
Probability(b)=0.83640081799591
Probability(c)=0.14110429447852763
Probability(d)=0.05112474437627812
Probability(e)=0.0081799591002045
